<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/Database_09_PokemonAndPostgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is PostgreSQL? How Does it Differ from SQLite?

PostgreSQL and SQLite are both relational database management systems that support ACID (Atomicity, Consistency, Isolation, Durability) properties and transactions. However, they have significant differences in their architectures, feature sets, and use cases.

**SQLite** is a lightweight, serverless, and self-contained database engine. It's ideal for small to medium-scale applications, embedded systems, or local data storage. SQLite stores the entire database as a single file on disk, making it easy to set up and manage. It's often used in mobile apps, desktop applications, and small websites.

On the other hand, **PostgreSQL** is a full-featured, server-based RDBMS designed to handle large amounts of data and support multiple concurrent users. Its client-server architecture allows it to manage resources more efficiently and handle heavier workloads.

When deciding between SQLite and PostgreSQL, consider the following factors:

1.  **Scalability**: As your application grows and the number of concurrent users increases, PostgreSQL's client-server architecture becomes crucial. It can handle a large number of simultaneous connections and efficiently manage resources. SQLite, being serverless, may struggle with high levels of concurrency and may not be suitable for applications with a large number of concurrent writers.
2.  **Data Size and Distribution**: PostgreSQL is designed to handle large and massive datasets, even in the terabyte range. It offers features like table partitioning, which allows you to split large tables across multiple files or even servers, improving query performance and manageability. SQLite, while capable of handling moderately sized datasets, may not be the best choice for extremely large or distributed datasets.
3.  **Advanced Features**: PostgreSQL offers a rich set of advanced features that become increasingly important as your application grows. These include:
    -   **Strict Typing**: PostgreSQL enforces strict data typing, ensuring data integrity and reducing the chances of data inconsistencies. This becomes increasingly critical when there many "writers" to the database.
    -   **Complex Queries**: PostgreSQL supports complex queries, including advanced joins, subqueries, and window functions, which are essential for handling sophisticated data retrieval tasks.
    -   **Stored Procedures and Triggers**: PostgreSQL allows you to define stored procedures and triggers, enabling you to encapsulate complex business logic within the database itself. This can lead to better performance and maintainability.
    -   **Extensibility**: PostgreSQL is highly extensible, allowing you to add custom data types, functions, and even programming languages. This flexibility becomes crucial as your application's requirements evolve.
    - **Security and User Management**: Postgres has built-in support for things like encryption, user management, password hashing, and other security measures. SQLite, by contrast, relies on the surrounding "application" (written in Python, Java, C#, etc.) to handle these things. This can be become impractical as the numbers of users becomes large.
4.  **Replication and High Availability**: As your application becomes mission-critical, you may need to ensure high availability and minimize downtime. PostgreSQL offers built-in replication features, such as streaming replication and logical replication, which allow you to create standby servers and distribute the workload. SQLite, being a serverless database, does not have built-in replication capabilities.
5. **Available Resources.** Postgres requires more physical resources (processing power, disk space) and human resources (e.g., a trained database administrator) than SQLite. SQLite's dynamic typing can make database development and deployment quicker than Postgres's strict typing. (In fact, SQLite is often used to develop protoype databases, which can then be "scaled up" to Postgres or a similar RDBMS).

While SQLite is a great choice for small to medium-sized applications, embedded systems, or local data storage, it may not be suitable for large-scale, high-concurrency, or mission-critical applications. In these cases, PostgreSQL's robustness, scalability, and advanced features make it the better choice.

For example, a large institution like a university or a financial organization would likely choose PostgreSQL over SQLite due to its ability to handle large amounts of data, support multiple concurrent users, and provide advanced features necessary for complex data management tasks.

## Data Types in Postgres

PostgreSQL offers a rich set of data types, including several that are not available in SQLite. These data types allow you to store and manipulate data more efficiently and with greater precision. Let's explore some of the key data types in PostgreSQL.

| Data Type Category | Examples | Description |
| --- | --- | --- |
| Numeric Types | `INTEGER`, `BIGINT`, `SMALLINT`, `DECIMAL`, `NUMERIC`, `REAL`, `DOUBLE PRECISION`, `SERIAL`, `BIGSERIAL` | Whole numbers, fixed-point numbers, floating-point numbers, and auto-incrementing integers. |
| Character Types | `CHAR(n)`, `VARCHAR(n)`, `TEXT` | Fixed-length and variable-length character strings. |
| Date/Time Types | `DATE`, `TIME`, `TIMESTAMP`, `INTERVAL` | Stores date, time, timestamp, and interval values. |
| Boolean Type | `BOOLEAN` | Stores a logical value of either `TRUE` or `FALSE`. |
| Enumerated Type | `ENUM` | Defines a custom data type with a static set of values. |
| Array Type | Any data type followed by `[]` | Represents an array of elements of the same type. |
| UUID Type | `UUID` | Stores Universally Unique Identifiers (UUIDs). |
| JSON and JSONB Types | `JSON`, `JSONB` | Stores JSON data as text or in a binary format. |
| Hstore Type | `HSTORE` | Represents a key-value pair data type. |
| Range Types | `INT4RANGE`, `TSRANGE`, `DATERANGE`, etc. | Represent a range of values, such as integers, timestamps, or dates. |

Here are a few examples to illustrate the usage of some of these data types:

1.  Enumerated Type:

```sql
CREATE TYPE mood AS ENUM ('sad', 'ok', 'happy');
```

2.  Array Type:

```sql
CREATE TABLE scores (
      id SERIAL PRIMARY KEY,
      student_id INTEGER,
      grades INTEGER[]
    );
```

When choosing data types for your database schema, consider factors such as data integrity, storage efficiency, and the nature of the data being stored. PostgreSQL's wide range of data types gives you flexibility and power in designing your database schema.

It's worth noting that while SQLite also supports many of these data types, such as numeric, character, and date/time types, it lacks some of the more advanced types like arrays, UUIDs, and range types. PostgreSQL's extensive set of data types is one of the factors that make it a more versatile and feature-rich database system.

In [1]:
# Insteall postgres
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%sql postgresql+psycopg2://@/postgres

 * Starting PostgreSQL 14 database server
   ...done.
CREATE ROLE


## Overview of the Pokemon Clinic Database

To demonstrate the power of Postgres, we'll be creating a The Pokemon Clinic database consists of three main tables: `trainers`, `pokemon`, and `medical_records`. These tables are designed to store information about Pokemon trainers, their Pokemon, and the medical records associated with each Pokemon.

In [2]:
%%sql
-- Create the trainers table
DROP TABLE IF EXISTS trainers CASCADE;
DROP TABLE IF EXISTS pokemon CASCADE;
DROP TABLE IF EXISTS medical_records;

-- Create the trainers table
CREATE TABLE trainers (
  id SERIAL PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  email VARCHAR(100) UNIQUE NOT NULL,
  password_hash VARCHAR(100) NOT NULL,
  phone VARCHAR(20),
  date_of_birth DATE,
  CHECK (date_of_birth < CURRENT_DATE)
);

-- Create the pokemon table
CREATE TABLE pokemon (
  id SERIAL PRIMARY KEY,
  name VARCHAR(50) NOT NULL,
  species VARCHAR(50) NOT NULL,
  trainer_id INTEGER REFERENCES trainers(id),
  level INTEGER CHECK (level BETWEEN 1 AND 100),
  health_status VARCHAR(20),
  abilities TEXT[]
);

-- Create the medical_records table
CREATE TABLE medical_records (
  id SERIAL PRIMARY KEY,
  pokemon_id INTEGER REFERENCES pokemon(id),
  visit_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  diagnosis TEXT,
  treatment TEXT,
  cost NUMERIC(8, 2)
);

 * postgresql+psycopg2://@/postgres
Done.
Done.
Done.
Done.
Done.
Done.


[]

While much of this statement should be familiar, you might notice a few new things, as well.

The `trainers` table stores information about Pokemon trainers.
  -   It includes columns for the trainer's ID (auto-generated), name, email (unique), hashed password, phone number, and date of birth.
  -   The `date_of_birth` column has a CHECK constraint that ensures the value is always less than the current date. This constraint uses the `CURRENT_DATE` function, which is specific to PostgreSQL and not available in SQLite.
  -   The `email` column has a UNIQUE constraint to ensure that each email address is associated with only one trainer.

The pokemon` table stores information about individual Pokemon.
  -   It includes columns for the Pokemon's ID (auto-generated), name, species, trainer ID (foreign key referencing the `trainers` table), level, health status, and abilities.
   -   The `level` column has a CHECK constraint that ensures the value is between 1 and 100.
  -   The `abilities` column is of type TEXT[] (array), which allows storing multiple abilities for each Pokemon. Arrays are a feature specific to PostgreSQL and not available in SQLite.

The `medical_records` table stores information about medical visits and treatments for each Pokemon.
  -   It includes columns for the medical record ID (auto-generated), Pokemon ID (foreign key referencing the `pokemon` table), visit date, diagnosis, treatment, and cost.
  -   The `visit_date` column has a DEFAULT value of `CURRENT_TIMESTAMP`, which automatically sets the value to the current timestamp if no value is provided during insertion. This is a PostgreSQL-specific feature.
  -   The `cost` column is of type NUMERIC(8, 2), which allows storing monetary values with a precision of 8 digits and 2 decimal places. The NUMERIC type provides more precise decimal calculations compared to SQLite's REAL type.

### Test Data for Pokemon Clinic
Now, let's insert some test data for our Pokemon Clinic

In [3]:
%%sql
CREATE EXTENSION IF NOT EXISTS pgcrypto;
-- Insert sample data into the trainers table
INSERT INTO trainers (name, email, password_hash, phone, date_of_birth)
VALUES
  ('Ash Ketchum', 'ash@example.com', crypt('bad_password', gen_salt('bf')), '123-456-7890', '1990-05-22'),
  ('Misty', 'misty@example.com', crypt('long_password_with_symbols_23423odsgv*732x', gen_salt('bf')), '987-654-3210', '1991-07-19');

-- Insert sample data into the pokemon table
INSERT INTO pokemon (name, species, trainer_id, level, health_status, abilities)
VALUES
  ('Pikachu', 'Pikachu', 1, 25, 'Healthy', ARRAY['Static', 'Lightning Rod']),
  ('Charizard', 'Charizard', 1, 65, 'Healthy', ARRAY['Blaze']),
  ('Staryu', 'Staryu', 2, 30, 'Injured', ARRAY['Natural Cure', 'Illuminate']);

-- Insert sample data into the medical_records table
INSERT INTO medical_records (pokemon_id, diagnosis, treatment, cost)
VALUES
  (3, 'Minor abrasions', 'Applied healing salve', 50.00),
  (1, 'Regular check-up', 'None', 0.00);

 * postgresql+psycopg2://@/postgres
Done.
2 rows affected.
3 rows affected.
2 rows affected.


[]

### `Trainers` Table: What is a Password "Hash"?
Again, much of the above `INSERT` statement should be familar. However, if you look closely at the ways the trainer's passwords are handled, you'll notice something a bit different.

- The `password_hash` column stores the hashed version of the trainer's password. **Hashing** is a one-way process that converts the plain-text password into a fixed-size string of characters. The resulting hash is irreversible, meaning it is computationally infeasible to obtain the original password from the hash.
- In this example, the `crypt` function is used along with the `gen_salt` function to hash the passwords. The `crypt` function applies a cryptographic hash function to the password, and the `gen_salt` function generates a random salt value using the Blowfish ('bf') algorithm. The salt is appended to the password before hashing, making it more resistant to rainbow table attacks and increasing the security of stored passwords.
- Storing hashed passwords instead of plain-text passwords is crucial for security. If the database is compromised, attackers would only have access to the hashed passwords, making it extremely difficult for them to retrieve the original passwords.

It's important to note that while the example uses the `crypt` function for simplicity, in a production environment, it's recommended to use more secure and modern hashing algorithms specifically designed for password hashing, such as bcrypt, scrypt, or PBKDF2, which provide better protection against various types of attacks.

We see what this hash looks like:

In [4]:
%%sql
SELECT * FROM trainers;

 * postgresql+psycopg2://@/postgres
2 rows affected.


id,name,email,password_hash,phone,date_of_birth
1,Ash Ketchum,ash@example.com,$2a$06$V6mJFWDR3jLM74O2syFUi.ECuvCkhSqHNYa0VJuJWV7MJ2Rk36l6.,123-456-7890,1990-05-22
2,Misty,misty@example.com,$2a$06$WW7XkylwYa/xXy8ay/OAJeniFtUzFKbQp5cZONHYUaXsFDZMx/yXG,987-654-3210,1991-07-19


If you look closely, you'll notice that the **password hash** is ALWAYS the exact same length, regardless of the initial length of the password. So, for example, if take two passwords, one of which is "123", and the other of which is the text of my favorite novel (300 pages--also a bad password, though for different reasons!), they will oth generate a "hash" of the exact same length.

The basic idea is this:
1. When the user first creates their password, we "hash" the password and store that hash (not the password) in the database.
2. WHen the user logs in again and enters their password again, we again "hash" whatever they entered, and compare this to our database. If it matches, we let them in!
3. The advantage of this is that if someone manages to break into our database and access the password hash, they won't be able to recover the user's password. This is because hashing is a **one-way** function. If you know the password, you can get the hash, but knowing the hash does NOT allow you to compute the password.



### Postgres Arrays in the `Pokemon` Table

Let's now take a look at the `pokemon` table, which has an array.

In [5]:
%%sql
SELECT * FROM pokemon;

 * postgresql+psycopg2://@/postgres
3 rows affected.


id,name,species,trainer_id,level,health_status,abilities
1,Pikachu,Pikachu,1,25,Healthy,"['Static', 'Lightning Rod']"
2,Charizard,Charizard,1,65,Healthy,['Blaze']
3,Staryu,Staryu,2,30,Injured,"['Natural Cure', 'Illuminate']"


Here, you'll notice there is an array of abilities. We can access this as follows:

In [6]:
%%sql
SELECT
  name,
  abilities[1] as first_ability,
  abilities[2] as second_ability,
  abilities[3] as third_ability
FROM pokemon;

 * postgresql+psycopg2://@/postgres
3 rows affected.


name,first_ability,second_ability,third_ability
Pikachu,Static,Lightning Rod,None
Charizard,Blaze,None,None
Staryu,Natural Cure,Illuminate,None


THe attempt to access the third ability fails (since the Pokemon don't have one!). However, it doesn't crash the database--it just returns `None`.

## A Better ALTER TABLE

In PostgreSQL, the `ALTER TABLE` command is more versatile and feature-rich compared to SQLite. While SQLite supports basic table modifications, PostgreSQL offers a wide range of options to alter tables efficiently. Let's explore some of the key improvements in PostgreSQL's `ALTER TABLE` command.

###  Adding Columns with Default Values
In PostgreSQL, you can add a new column to a table and specify a default value for existing rows in a single statement. Example:

In [7]:
%%sql
ALTER TABLE trainers ADD COLUMN created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP;

 * postgresql+psycopg2://@/postgres
Done.


[]

This statement adds a new column named `created_at` of type `TIMESTAMP` to the `trainers` table and sets the default value to the current timestamp for existing rows.

### Modifying Column Data Types
PostgreSQL allows you to modify the data type of a column using the `ALTER TABLE` command. Example:

In [8]:
%%sql
ALTER TABLE trainers ALTER COLUMN phone TYPE VARCHAR(15);

 * postgresql+psycopg2://@/postgres
Done.


[]

This statement changes the data type of the `phone` column in the `trainers` table from `VARCHAR(20)` to `VARCHAR(15)`.

### Adding Constraints
PostgreSQL enables you to add constraints to existing tables using the `ALTER TABLE` command. Example:

In [9]:
%%sql
ALTER TABLE pokemon ADD CONSTRAINT unique_name_per_trainer UNIQUE (name, trainer_id);

 * postgresql+psycopg2://@/postgres
Done.


[]

This statement adds a unique constraint named `unique_name_per_trainer` to the `pokemon` table, ensuring that the combination of `name` and `trainer_id` is unique.

These examples demonstrate the powerful capabilities of PostgreSQL's `ALTER TABLE` command. PostgreSQL provides a rich set of options to modify table structures, add or drop constraints, change column data types, and perform various other table alterations.

The flexibility and extensive features of `ALTER TABLE` in PostgreSQL allow for efficient schema modifications without the need to recreate tables from scratch. This is particularly useful in scenarios where the database schema needs to evolve over time to accommodate changing requirements.

It's important to note that while SQLite does support some basic `ALTER TABLE` operations, such as renaming tables and adding columns, it lacks the extensive options and flexibility provided by PostgreSQL's `ALTER TABLE` command.

## Stored Procedures in PostgreSQL

In the world of Pokemon, trainers often have a set of routines they follow to care for their Pokemon. They may have a specific way of feeding them, training them, or even a special technique for battling. These routines help trainers manage their Pokemon effectively and consistently. Similarly, in the world of databases, we have stored procedures that allow us to encapsulate and reuse common tasks or complex operations.

### What are Stored Procedures?

A **stored procedure** is a precompiled collection of SQL statements and optional control-flow statements stored under a name and processed as a unit. It is a database object that performs a specific task or a series of tasks when invoked. Stored procedures are stored within the database itself and can be called or executed whenever needed.

Stored procedures offer several benefits:

1.  *Reusability*. Stored procedures can be called multiple times from different parts of an application, promoting code reuse and reducing duplication.
2.  *Encapsulation*. Stored procedures encapsulate complex logic and SQL statements, making the code more modular and easier to maintain.
3.  *Performance*. Since stored procedures are precompiled and stored in the database, they can execute faster compared to individual SQL statements sent from an application.
4.  *Security*. Stored procedures can help enforce security by granting execute permissions to users without giving them direct access to the underlying tables.

### Basic Syntax of Stored Procedures

In PostgreSQL, you can create a stored procedure using the `CREATE PROCEDURE` statement. Here's the basic syntax:

```sql
CREATE PROCEDURE procedure_name(parameter1 datatype, parameter2 datatype, ...)
AS $$
BEGIN
  -- Procedure logic goes here
  -- SQL statements and control-flow statements
END;
$$ LANGUAGE plpgsql;
```

Let's break down the syntax:

-   `procedure_name`: The name you give to the stored procedure.
-   `parameter1`, `parameter2`, etc.: Optional input parameters that the procedure can accept. You specify the parameter name and its data type.
-   `AS $$`: Indicates the start of the procedure body.
-   `BEGIN` and `END`: Delimits the procedure body, which contains the SQL statements and control-flow statements.
-   `LANGUAGE plpgsql`: Specifies the language used for the stored procedure, in this case, PL/pgSQL (Procedural Language/PostgreSQL).

### Example: Stored Procedure for the Pokemon Clinic

Let's create a stored procedure for the Pokemon Clinic that retrieves the medical records of a specific Pokemon based on its ID.

In [26]:
%%sql
CREATE OR REPLACE PROCEDURE update_pokemon_level(
  p_pokemon_id INTEGER,
  p_new_level INTEGER
)
AS $$
BEGIN
  -- Update the level of the Pokemon
  UPDATE pokemon
  SET level = p_new_level
  WHERE id = p_pokemon_id;

  -- Check if any rows were affected
  IF FOUND THEN
    RAISE NOTICE 'Pokemon level updated successfully';
  ELSE
    RAISE NOTICE 'No Pokemon found with the given ID';
  END IF;
END;
$$ LANGUAGE plpgsql;

 * postgresql+psycopg2://@/postgres
Done.


[]

In this example:

-   The `CREATE OR REPLACE PROCEDURE` statement is used to create a procedure named `update_pokemon_level`.
-   The procedure takes two input parameters: `p_pokemon_id` (INTEGER) representing the ID of the Pokemon to update, and `p_new_level` (INTEGER) representing the new level to set for the Pokemon.
-   Inside the procedure body, an `UPDATE` statement is used to update the `level` column of the `pokemon` table for the row where the `id` matches the provided `p_pokemon_id`.
-   The `IF FOUND THEN` clause checks if any rows were affected by the `UPDATE` statement. If rows were affected (i.e., a Pokemon with the given ID was found and updated), it raises a notice indicating that the level was updated successfully (it also **logs** this notice). Otherwise, it raises a notice indicating that no Pokemon was found with the given ID.

To call this stored procedure and update the level of a Pokemon, you can use the `CALL` statement followed by the procedure name and the required arguments. For example:

In [27]:
%%sql
CALL update_pokemon_level(1, 30);

 * postgresql+psycopg2://@/postgres
Done.


[]

This statement calls the `update_pokemon_level` procedure, passing the values `1` as the `p_pokemon_id` parameter and `30` as the `p_new_level` parameter. The procedure will then execute the `UPDATE` statement and update the level of the Pokemon with ID 1 to 30.

## User Management and Roles in PostgreSQL

When it comes to managing a database, one of the most critical aspects is ensuring the security and integrity of the data. In the world of Pokemon, this means safeguarding sensitive information about trainers, their Pokemon, and the medical records of the Pokemon Clinic. Just like how a Pokemon trainer must carefully manage their team and delegate responsibilities, a database administrator must effectively manage user access and privileges to maintain a secure and organized system.

PostgreSQL provides a robust user management system that allows you to create, modify, and delete user accounts, as well as assign privileges and roles to users. By properly setting up user accounts and roles, you can ensure that each user has access to only the necessary data and functionality, reducing the risk of unauthorized access or data breaches.

Let's explore how to manage users and their roles in PostgreSQL using our Pokemon Clinic database as an example.

### Creating Users
In PostgreSQL, you can create a new user using the `CREATE USER` statement. For example, let's create a user account for the head nurse of the Pokemon Clinic:

In [10]:
%%sql
CREATE USER nurse_joy WITH PASSWORD 'chansey123';

 * postgresql+psycopg2://@/postgres
Done.


[]

This statement creates a new user named `nurse_joy` with the password `'chansey123'`. It's important to choose a strong and secure password to protect the user account from unauthorized access.

### Creating Roles
**Roles** in PostgreSQL are used to group together privileges and assign them to users. Think of roles as job titles or categories that define what actions users can perform. Let's create a role for Pokemon trainers:

In [11]:
%%sql
CREATE ROLE pokemon_trainer;

 * postgresql+psycopg2://@/postgres
Done.


[]

This statement creates a new role named `pokemon_trainer`. We can later assign this role to users who are authorized to perform actions related to Pokemon training.

### Granting Privileges to Roles
Once you have created a role, you can grant specific privileges to that role using the `GRANT` statement. For example, let's grant the necessary privileges to the `pokemon_trainer` role:

In [12]:
%%sql
GRANT SELECT, INSERT, UPDATE ON pokemon TO pokemon_trainer;

 * postgresql+psycopg2://@/postgres
Done.


[]

### Assigning Roles to Users
To assign a role to a user, you can use the GRANT statement with the TO clause. Let's assign the pokemon_trainer role to a user named ash_ketchum:

In [13]:
%%sql
CREATE USER ash_ketchum WITH PASSWORD 'pika!';
GRANT pokemon_trainer TO ash_ketchum;

 * postgresql+psycopg2://@/postgres
Done.
Done.


[]

This statement grants the `pokemon_trainer` role to the user `ash_ketchum`. Now, `ash_ketchum` has all the privileges associated with the `pokemon_trainer` role.

### Revoking Privileges and Roles
Sometimes, you may need to revoke privileges or roles from users or roles. This is useful when a user's responsibilities change or when you want to restrict access to certain data. You can use the `REVOKE` statement to revoke privileges and roles. For example:

In [14]:
%%sql
REVOKE UPDATE ON pokemon FROM pokemon_trainer;
REVOKE pokemon_trainer FROM ash_ketchum;

 * postgresql+psycopg2://@/postgres
Done.
Done.


[]

The first statement revokes the `UPDATE` privilege on the `pokemon` table from the `pokemon_trainer` role. This means that users with the `pokemon_trainer` role will no longer be able to modify data in the `pokemon` table. The second statement revokes the `pokemon_trainer` role from the user `ash_ketchum`. As a result, `ash_ketchum` will no longer have the privileges associated with the `pokemon_trainer` role.

PostgreSQL allows you to modify user attributes and delete user accounts using the `ALTER USER` and `DROP USER` statements, respectively. For example:

In [15]:
%%sql
ALTER USER nurse_joy WITH PASSWORD 'blissey456';
DROP USER ash_ketchum;

 * postgresql+psycopg2://@/postgres
Done.
Done.


[]

The first statement changes the password of the user `nurse_joy` to `'blissey456'`. It's a good practice to regularly update passwords to enhance security. The second statement deletes the user `ash_ketchum` from the database. This should be done with caution and only when necessary, as it permanently removes the user and their associated permissions.


### Importance of User Management and Security

Proper user management is crucial for maintaining the security of your database. By creating separate user accounts and assigning appropriate privileges and roles, you can ensure that users have access only to the necessary data and operations. Some important security considerations include:
  -   Use strong and unique passwords for user accounts.
  -   Grant privileges based on the **principle of least privilege**, giving users only the permissions they need to perform their tasks.
  -   Regularly review and audit user privileges to ensure they align with the users' responsibilities.
  -   Revoke unnecessary privileges and remove inactive user accounts to minimize security risks.

By leveraging PostgreSQL's user management and role-based access control features, you can effectively manage user access, enforce security policies, and protect sensitive data in your database.

##  Deploying Your PostgreSQL Database

Congratulations on learning the fundamentals of PostgreSQL! You now have the skills to create tables, insert data, and manage users and their roles. The next step is to deploy your database so that it can be accessed by your application. When it comes to deploying PostgreSQL, you have two main options: on-site deployment and cloud-based deployment. In this section, we'll explore each option in detail, using our Pokemon Clinic database as an example.

### On-site Deployment

On-site deployment, also known as on-premises deployment, means installing and running PostgreSQL on your own physical servers or computers within your organization's network.

Imagine you're setting up the database for the Pokemon Clinic. With on-site deployment, you would need to buy or use existing **servers** (powerful computers designed to host applications and databases) to run PostgreSQL. You'll also need to configure the operating system (the software that manages the computer's hardware and software resources) and set up security measures to protect the database. While SQLite is often "built in" to programming languages like Python, Postres needs to be downloaded and installed for your particular operating system

Some advantages of On-site deployment include

1.  *Control*. On-site deployment gives you full control over your PostgreSQL environment. You can customize and optimize it to meet your specific needs.
2.  *Security* With on-site deployment, you have complete control over the security of your database. This is particularly important for the Pokemon Clinic, as it stores sensitive information about Pokemon and their trainers.

Some disadvantages include:

1.  *Cost*. On-site deployment can be expensive, as you need to invest in hardware, software licenses, and infrastructure setup. You'll also need to factor in the ongoing costs of maintenance and support.
2.  *Maintenance*. With on-site deployment, you're responsible for maintaining the database. This includes tasks like installing updates, making backups, and troubleshooting issues. You'll need a dedicated team or staff member with the necessary expertise to handle these tasks.

### Cloud-based Deployment

Cloud-based deployment involves running PostgreSQL on servers provided by a third-party vendor, such as Amazon Web Services (AWS), Google Cloud Platform (GCP), or Microsoft Azure.

In the context of the Pokemon Clinic, you would set up your database on a cloud platform, and the cloud provider would take care of the underlying infrastructure, installation, and configuration of PostgreSQL.

The main advantages of cloud-based deployment include:

1.  *Scalability*. Cloud platforms allow you to easily scale your PostgreSQL database up or down based on your needs. For example, if the Pokemon Clinic experiences a surge in patients, you can quickly allocate more resources to the database to handle the increased load.
2.  *Cost-effective*. With cloud-based deployment, you only pay for the resources you use. This can be more cost-effective than on-site deployment, where you need to invest in expensive hardware upfront.
3.  *Maintenance*. Cloud providers take care of routine maintenance tasks, such as software updates and backups. This frees up your time and resources to focus on running the Pokemon Clinic.

Disadvantages of cloud-based deployment include:

1.  *Security*. When you deploy your database on a cloud platform, you're trusting the cloud provider with your data. While cloud providers have robust security measures in place, some organizations may prefer (or be required by law) to keep sensitive data on-site.
2.  *Dependency*. With cloud-based deployment, you're dependent on the cloud provider. If the cloud provider experiences an outage or issue, it could impact your database and application.

### Choosing the Right Deployment Option

When choosing between on-site and cloud-based deployment for your PostgreSQL database, consider the following factors:

1.  *Security/Legal requirements*. If you're storing sensitive data, like the Pokemon Clinic's medical records, you may prefer on-site deployment for added control over security.
2.  *Scalability needs*. If you anticipate needing to quickly scale your database up or down, cloud-based deployment may be the better choice.
3.  *Budget*. On-site deployment requires a larger upfront investment, while cloud-based deployment allows you to pay for only the resources you use. Consider your budget and long-term costs when making a decision.
4.  *Maintenance capabilities*. If you have a dedicated IT team that can handle database maintenance, on-site deployment may be a good fit. If you prefer to focus on your core business, cloud-based deployment can free up your resources.

In real life, you may also opt for a **hybrid** option, where some databases live on the "cloud", while others like locally.


## Other Relational Database Management Systems (RDBMSs)

While SQLite and PostgreSQL are popular choices for relational database management, there are several other RDBMSs available in the market. Each RDBMS has its own strengths, features, and target audiences. Let's take a brief look at some of the other notable RDBMSs and compare them to SQLite and PostgreSQL.

### MySQL and MariaDB

MySQL is an open-source RDBMS (now owned by Oracle) that is widely used for web applications and online services. It is known for its simplicity, ease of use, and good performance for read-heavy workloads. MySQL supports various storage engines, including InnoDB and MyISAM, which offer different features and trade-offs. Compared to PostgreSQL, MySQL is sometimes considered to be simpler and easier to set up, making it a popular choice for small to medium-sized applications. However, PostgreSQL tends to have more advanced features and is often preferred for complex queries and data integrity. **Maria DB** is an off-shoot of MySQL that implements many of the same features.

### Oracle Database

Oracle Database is a commercial RDBMS that is widely used in enterprise environments. It is known for its scalability, reliability, and advanced features. Oracle Database provides a comprehensive set of tools for data management, including partitioning, data replication, and advanced security features. It supports a wide range of data types and has a powerful optimizer for query performance. Compared to SQLite and PostgreSQL, Oracle Database is generally more complex and requires more resources to set up and maintain. It is typically used in large-scale, mission-critical applications where high performance and enterprise-level features are required.

### Microsoft SQL Server

Microsoft SQL Server is a commercial RDBMS developed by Microsoft. It is commonly used in Windows-based environments and integrates well with other Microsoft technologies. SQL Server provides a range of features, including support for T-SQL (Transact-SQL), a proprietary extension to standard SQL, and integration with Microsoft's business intelligence tools. It offers strong performance, scalability, and security features. Compared to SQLite and PostgreSQL, SQL Server is more tightly integrated with the Microsoft ecosystem and is often preferred by organizations using Microsoft technologies.

### IBM Db2

IBM Db2 is a commercial RDBMS developed by IBM. It is known for its scalability, reliability, and support for large-scale enterprise applications. Db2 provides advanced features such as data compression, data encryption, and multi-temperature data management. It is available on various platforms, including Linux, Unix, and Windows. Compared to SQLite and PostgreSQL, Db2 is generally more suited for large-scale, enterprise-level applications and is often used in mainframe environments.

When comparing these RDBMSs to SQLite and PostgreSQL, it's important to consider factors such as scalability, performance, features, ecosystem, and cost. SQLite is lightweight and embedded, making it suitable for small-scale applications or local data storage. PostgreSQL is known for its robustness, extensibility, and adherence to SQL standards, making it a good choice for applications that require advanced features and data integrity.

Ultimately, the choice of RDBMS depends on the specific requirements of your application, the scale of your data, the platform and ecosystem you are working with, and your budget. It's essential to evaluate the features, performance, and costs of each RDBMS and choose the one that aligns best with your needs.